In [1]:
import requests
from bs4 import BeautifulSoup
import re 
import pandas as pd

In [2]:
headers = {'User-Agent':
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.141 Safari/537.36'}
url = 'https://kolesa.kz/cars/shymkent/?year[from]=2010'

In [3]:
req = requests.get(url)

soup = BeautifulSoup(req.content, 'html.parser')

In [30]:
from lxml.html import fromstring
def get_proxies():
    url = 'https://free-proxy-list.net/'
    response = requests.get(url)
    parser = fromstring(response.text)
    proxies = set()
    for i in parser.xpath('//tbody/tr')[:50]:
        if i.xpath('.//td[7][contains(text(),"yes")]'):
            #Grabbing IP and corresponding PORT
            proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
            proxies.add(proxy)
    return proxies

In [32]:
from itertools import cycle
import traceback

proxies = get_proxies()

proxy_pool = cycle(proxies)

In [4]:
links = []
url = 'https://kolesa.kz/cars/shymkent/?year[from]=2010'
for q in range (1, 20):
    response = requests.get(url)
    #proxy = next(proxy_pool)
    
    #try:
     #   response = requests.get(url, proxies={"http": proxy, "https": proxy})
        
        
    #except:
     #   print("Skipping. Connnection error")
    
    soup = BeautifulSoup(response.content, 'html.parser')
    span = soup.find_all('span', class_ = 'a-el-info-title')
    for i in span:
        for j in i.find_all('a', class_ = 'list-link ddl_product_link'):
            links.append('https://kolesa.kz' + j['href'])
            
    url = soup.find('a', class_='right-arrow next_page')
    url = 'https://kolesa.kz' + url['href']
        
    
links = set(links)
print(len(links))

372


In [8]:
scrapped_data = []
dlist = ['Город', 'Кузов', 'Объем двигателя, л', 'Пробег', 'Коробка передач', 'Руль', 'Цвет', 'Привод', 'Растаможен в Казахстане']

for i in links:
    response = requests.get(i)
    
    if response:
        
        sp = BeautifulSoup(response.content, 'html.parser')
        
        dc = {k: 'NaN' for k in dlist}
    
        brand = sp.find('span', itemprop = 'brand')
        if brand:
            brand = brand.text.strip()
        
        elif brand == None:
            brand = 'NaN'
            
        name = sp.find('span', itemprop = 'name')
        if name:
            name = name.text.strip()
        
        elif name == None:
            name = 'NaN'
    
        year = sp.find('span', class_ = 'year')
        if year:
            year = year.text.strip()
        
        elif year == None:
            year = 'NaN'
            
        price = sp.find('div', class_ = 'offer__price').text.strip().replace('\n', '').replace('\xa0', '').replace(' ', '')
        price = price[:-1]
    
        currency = sp.find('div', class_ = 'offer__price').span.extract().text.strip().replace(' ', '')
    
        dd = []
        dt = []
    
        ddd = sp.find_all('dd')
        for i in ddd:
            dd.append(i.text.strip().replace('  ', '').replace('\n', ' '))
    
        ddt = sp.find_all('dt')
        for i in ddt:
            dt.append(i.text.strip().replace('  ', '').replace('\n', ' '))
    
        for i in range(len(dt)):
            for key in dc:
                if dt[i] == key:
                    dc[key] = dd[i]
    
        data = {
            'Brand' : brand,
            'Name' : name,
            'Year' : year,
            'Price' : price,
            'Currency' : currency
        }

        data.update(dc)

        scrapped_data.append(data)
    
    else:
        continue
        

In [18]:
r = requests.get('https://kolesa.kz/a/show/101950998')
sp = BeautifulSoup(r.content, 'html.parser')

dlist = ['Город', 'Кузов', 'Объем двигателя, л', 'Пробег', 'Коробка передач', 'Руль', 'Цвет', 'Привод', 'Растаможен в Казахстане']

dc = {k: 'NaN' for k in dlist}
    
brand = sp.find('span', itemprop = 'brand').text.strip()
    
name = sp.find('span', itemprop = 'name').text.strip()
    
year = sp.find('span', class_ = 'year').text.strip()
    
price = sp.find('div', class_ = 'offer__price').text.strip().replace('\n', '').replace('\xa0', '').replace(' ', '')
price = price[:-1]
    
currency = sp.find('div', class_ = 'offer__price').span.extract().text.strip().replace(' ', '')
    
    
dd = []
dt = []
    
ddd = sp.find_all('dd')
for i in ddd:
    dd.append(i.text.strip().replace('  ', '').replace('\n', ' '))
    
ddt = sp.find_all('dt')
for i in ddt:
    dt.append(i.text.strip().replace('  ', '').replace('\n', ' '))
    
for i in range(len(dt)):
    for key in dc:
        if dt[i] == key:
            dc[key] = dd[i]
        
data = {
    'Brand' : brand,
    'Name' : name,
    'Year' : year,
    'Price' : price,
    'Currency' : currency
}

data.update(dc)

data

{'Brand': 'Hyundai',
 'Name': 'Sonata',
 'Year': '2021',
 'Price': '9790000',
 'Currency': '₸',
 'Город': 'Шымкент',
 'Кузов': 'седан',
 'Объем двигателя, л': '2 (бензин)',
 'Пробег': '5 км',
 'Коробка передач': 'автомат',
 'Руль': 'слева',
 'Цвет': 'NaN',
 'Привод': 'передний привод',
 'Растаможен в Казахстане': 'Да'}

In [10]:
df = pd.DataFrame(scrapped_data)

In [11]:
df

,Brand,Name,Year,Price,Currency,Город,Кузов,"Объем двигателя, л",Пробег,Коробка передач,Руль,Цвет,Привод,Растаможен в Казахстане
0,ВАЗ (Lada),Largus,2013,3100000,₸,Шымкент,универсал,1.6 (газ-бензин),NaN,механика,слева,белый,передний привод,Да
1,Daewoo,Nexia,2012,2100000,₸,Шымкент,седан,1.6 (бензин),141 000 км,механика,слева,серебристый металлик,передний привод,Да
2,Mitsubishi,Outlander,2013,6500000,₸,Шымкент,кроссовер,2.4 (бензин),100 375 км,автомат,слева,черный металлик,полный привод,Да
3,УАЗ,Hunter,2017,4500000,₸,Шымкент,внедорожник,2.7 (бензин),55 000 км,механика,слева,NaN,полный привод,Да
4,ВАЗ (Lada),2190 (седан),2018,3650000,₸,Шымкент,седан,1.6 (бензин),23 500 км,механика,слева,серебристый металлик,передний привод,Да
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,Toyota,Camry,2017,13000000,₸,Шымкент,седан,2.5 (бензин),139 000 км,автомат,слева,белый металлик,передний привод,Да
368,Ravon,R4,2017,4250000,₸,Шымкент,седан,1.5 (бензин),49 500 км,механика,слева,белый металлик,передний привод,Да
369,Hyundai,Solaris,2011,2900000,₸,Шымкент,седан,1.4 (бензин),170 000 км,автомат,слева,серый металлик,передний привод,Нет
370,Renault,Kaptur,2020,9796000,₸,Шымкент,кроссовер,1.6 (бензин),NaN,механика,слева,NaN,передний привод,Да


In [12]:
df.to_excel("car_sells.xlsx", index = False, header = True)